# Run autoRIFT for coregistered SLCs

In [1]:
# Import required packages
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from datetime import datetime, timedelta
import time
import rasterio as rio
import xarray as xr
import rioxarray
import isce
import logging
root_logger = logging.getLogger()
root_logger.setLevel('WARNING')
import logging
from imageMath import IML
import isce
from components.contrib.geo_autoRIFT.autoRIFT import autoRIFT_ISCE
from components.contrib.geo_autoRIFT.autoRIFT import __version__ as version
import isceobj
import subprocess
from scipy.interpolate import interpn

In [2]:
os.environ['AUTORIFT'] = '/mnt/Backups/gbrench/sw/insar_tools/isce2/src/isce2/contrib/geo_autoRIFT'
os.environ['PYTHONPATH'] = os.environ['AUTORIFT']
os.environ['PATH'] += f":{os.environ['AUTORIFT']}"

In [3]:
scene_list=[
'S1A_IW_SLC__1SDV_20191004T121345_20191004T121412_029309_0354B0_F48E',
'S1A_IW_SLC__1SDV_20191016T121345_20191016T121412_029484_035ABB_373F',
'S1A_IW_SLC__1SDV_20191028T121345_20191028T121412_029659_0360C3_574E',
'S1A_IW_SLC__1SDV_20191109T121345_20191109T121412_029834_0366F3_462C',
'S1A_IW_SLC__1SDV_20191121T121344_20191121T121411_030009_036CFD_787C',
'S1A_IW_SLC__1SDV_20191203T121344_20191203T121411_030184_03730F_4C1A',
'S1A_IW_SLC__1SDV_20191215T121344_20191215T121411_030359_03791A_4B57',
'S1A_IW_SLC__1SDV_20191227T121343_20191227T121410_030534_037F23_3C47',
'S1A_IW_SLC__1SDV_20200108T121343_20200108T121410_030709_038533_C53D',
'S1A_IW_SLC__1SDV_20200120T121342_20200120T121409_030884_038B59_8828',
'S1A_IW_SLC__1SDV_20200201T121342_20200201T121409_031059_039174_1908',
'S1A_IW_SLC__1SDV_20200213T121341_20200213T121408_031234_039785_41EB',
'S1A_IW_SLC__1SDV_20200225T121341_20200225T121408_031409_039D91_8F87',
'S1A_IW_SLC__1SDV_20200308T121341_20200308T121408_031584_03A39B_6624',
'S1A_IW_SLC__1SDV_20200320T121342_20200320T121409_031759_03A9BC_08F7',
'S1A_IW_SLC__1SDV_20200401T121342_20200401T121409_031934_03AFE6_E721',
'S1A_IW_SLC__1SDV_20200413T121342_20200413T121409_032109_03B610_2EA6',
           ]

In [4]:
proc_path = '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12'

In [5]:
def select_pairs(scene_list, min_temp_bline, max_temp_bline):
    scene_dates = {}
    for scene in scene_list:
        date = scene[17:25]
        scene_dates[date] = scene

    pairs = []
    for date1 in scene_dates:
        for date2 in scene_dates:
            if datetime.strptime(date2, '%Y%m%d')-datetime.strptime(date1, '%Y%m%d') < timedelta(days=max_temp_bline) and not datetime.strptime(date2, '%Y%m%d')-datetime.strptime(date1, '%Y%m%d') < timedelta(days=min_temp_bline) and not date1 >= date2 :
                pairs.append([date1, date2])
    
    print(f'number of pairs: {len(pairs)}')
    
    return pairs

In [6]:
pairs = select_pairs(scene_list, 80, 360)

number of pairs: 55


In [7]:
def run_autoRIFT(pair, skip_x=3, skip_y=18, scale_y_chip=6, min_azm_chip=16, max_azm_chip=64,
                 preproc_filter_width=21, mpflag=10, search_limit_x=4, search_limit_y=20):

    print(f'opening pair: {pair[0]}-{pair[1]}')

    reference_fn = f'{proc_path}/work/merged_cslcs/SLC/{pair[0]}/{pair[0]}.slc.full'
    secondary_fn = f'{proc_path}/work/merged_cslcs/SLC/{pair[1]}/{pair[1]}.slc.full'

    ds1 = gdal.Open(reference_fn, gdal.GA_ReadOnly)
    slc1 = ds1.GetRasterBand(1).ReadAsArray()

    ds2 = gdal.Open(secondary_fn, gdal.GA_ReadOnly)
    slc2 = ds2.GetRasterBand(1).ReadAsArray()

    I1 = np.abs(slc1)
    I2 = np.abs(slc2)
        
    obj = autoRIFT_ISCE()
    obj.configure()
    obj.MultiThread = mpflag

    # rotate to take advantage of chip size scaling
    I1 = np.rot90(I1)
    I2 = np.rot90(I2)
    # scale range chip size to get nearly square chip in cartesian coordinates
    obj.ScaleChipSizeY = scale_y_chip

    obj.I1 = I1
    obj.I2 = I2

    obj.SkipSampleX = skip_x
    obj.SkipSampleY = skip_y

    # Kernel sizes to use for correlation
    obj.ChipSizeMinX = min_azm_chip
    obj.ChipSizeMaxX = max_azm_chip
    obj.ChipSize0X = min_azm_chip
    # oversample ratio, balancing precision and performance for different chip sizes
    obj.OverSampleRatio = {obj.ChipSize0X:32,obj.ChipSize0X*2:64,obj.ChipSize0X*4:128}

    # generate grid
    m,n = obj.I1.shape
    xGrid = np.arange(obj.SkipSampleX+10,n-obj.SkipSampleX,obj.SkipSampleX)
    yGrid = np.arange(obj.SkipSampleY+10,m-obj.SkipSampleY,obj.SkipSampleY)
    nd = xGrid.__len__()
    md = yGrid.__len__()
    obj.xGrid = np.int32(np.dot(np.ones((md,1)),np.reshape(xGrid,(1,xGrid.__len__()))))
    obj.yGrid = np.int32(np.dot(np.reshape(yGrid,(yGrid.__len__(),1)),np.ones((1,nd))))
    noDataMask = np.invert(np.logical_and(obj.I1[:, xGrid-1][yGrid-1, ] > 0, obj.I2[:, xGrid-1][yGrid-1, ] > 0))

    # set search limits
    obj.SearchLimitX = np.full_like(obj.xGrid, search_limit_x)
    obj.SearchLimitY = np.full_like(obj.xGrid, search_limit_y)

    # set search limit and offsets in nodata areas
    obj.SearchLimitX = obj.SearchLimitX * np.logical_not(noDataMask)
    obj.SearchLimitY = obj.SearchLimitY * np.logical_not(noDataMask)
    obj.Dx0 = obj.Dx0 * np.logical_not(noDataMask)
    obj.Dy0 = obj.Dy0 * np.logical_not(noDataMask)
    obj.Dx0[noDataMask] = 0
    obj.Dy0[noDataMask] = 0
    obj.NoDataMask = noDataMask

    print("preprocessing")
    obj.WallisFilterWidth = preproc_filter_width
    obj.preprocess_filt_hps()
    obj.uniform_data_type()

    print("starting autoRIFT")
    obj.runAutorift()
    print("autoRIFT complete")

    # return outputs to original orientation
    tmpDx = np.rot90(obj.Dx, axes=(1, 0))
    obj.Dx = np.rot90(obj.Dy, axes=(1, 0))
    obj.Dy = tmpDx
    tmpxGrid = np.rot90(obj.xGrid, axes=(1, 0))
    obj.xGrid = np.rot90(obj.yGrid, axes=(1, 0))
    obj.yGrid = tmpxGrid
    obj.InterpMask = np.rot90(obj.InterpMask, axes=(1, 0))
    obj.ChipSizeX = np.rot90(obj.ChipSizeX, axes=(1, 0))
    obj.NoDataMask = np.rot90(obj.NoDataMask, axes=(1, 0))

    # convert displacement to m
    obj.Dx_m = obj.Dx * 2.3
    obj.Dy_m = obj.Dy * 12.1
        
    return obj

In [8]:
os.makedirs(f'{proc_path}/offsets', exist_ok=True)

In [9]:
# for i, pair in enumerate(pairs):
#     print(f'working on {i+1}/{len(pairs)}')
#     if not os.path.exists(f'{proc_path}/offsets/{pair[0]}-{pair[1]}.nc'):
#         obj = run_autoRIFT(pair)
#         # mask nodata:
#         obj.Dx_m[obj.NoDataMask[0:-1, 1:-1]] = np.nan
#         obj.Dy_m[obj.NoDataMask[0:-1, 1:-1]] = np.nan
    
#         # load to xarray 
#         ds = xr.Dataset({'Dx_m':(['y', 'x'], obj.Dx_m),
#                          'Dy_m':(['y', 'x'], obj.Dy_m),
#                          'chip_size_azm':(['y', 'x'], obj.ChipSizeX),
#                          'interpMask':(['y', 'x'], obj.InterpMask)},
#                         {'range':obj.xGrid[0, :],
#                          'azimuth':obj.yGrid[:, 0]})
        
#         # save tif for mintpy
#         ds.Dx_m.rio.to_raster(f'{proc_path}/offsets/dx_m_{pair[0]}-{pair[1]}.tif')
#         ds.to_netcdf(f'{proc_path}/offsets/{pair[0]}-{pair[1]}.nc')
#     else:
#         print('autoRIFT outputs exist, skipping')
#     print('--------------------------------')

In [10]:
for i, pair in enumerate(pairs):
    print(f'working on {i+1}/{len(pairs)}')
    if not os.path.exists(f'{proc_path}/offsets/{pair[0]}-{pair[1]}.nc'):
        obj = run_autoRIFT(pair)

        # interpolate to original dimensions 
        x_coords = np.flip(obj.xGrid[0, :])
        y_coords = obj.yGrid[:, 0]
        
        # Create a mesh grid for the slc dimensions
        x_coords_new, y_coords_new = np.meshgrid(
            np.arange(obj.I2.shape[0]),
            np.arange(obj.I2.shape[1])
        )

        # Perform bilinear interpolation using scipy.interpolate.interpn
        Dx_full = interpn((y_coords, x_coords), obj.Dx_m, (y_coords_new, x_coords_new), method="linear", bounds_error=False)
            
        # load to xarray 
        ds = xr.Dataset({'Dx_m':(['y', 'x'], Dx_full)},
                        {'range':x_coords_new[0, :],
                         'azimuth':y_coords_new[:, 0]})

        # multilook to match igram
        da_multilooked = ds.Dx_m.coarsen(x=5, boundary='trim').mean()
        
        # save tif for mintpy
        da_multilooked.rio.to_raster(f'{proc_path}/offsets/dx_m_{pair[0]}-{pair[1]}.tif')
    else:
        print('autoRIFT outputs exist, skipping')
    print('--------------------------------')

working on 1/55
opening pair: 20191004-20191227
preprocessing
starting autoRIFT


/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1468: RuntimeWarning: divide by zero encountered in divide
  DxMadmin = np.ones(Dx.shape) / OverSampleRatio / SearchLimitX * 2;
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1469: RuntimeWarning: divide by zero encountered in divide
  DyMadmin = np.ones(Dy.shape) / OverSampleRatio / SearchLimitY * 2;
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1413: RuntimeWarning: All-NaN slice encountered
  D = np.nanmedian(B,axis=0)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1416: RuntimeWarning: All-NaN slice encountered
  C = np.nanmedian(D,axis=0, overwrite_input=True)
/mnt/Backups/gbrench/sw/miniconda3/e

autoRIFT complete


/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


--------------------------------
working on 2/55
opening pair: 20191004-20200108
preprocessing
starting autoRIFT


...

--------------------------------
working on 55/55
opening pair: 20200120-20200413
preprocessing
starting autoRIFT


/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1468: RuntimeWarning: divide by zero encountered in divide
  DxMadmin = np.ones(Dx.shape) / OverSampleRatio / SearchLimitX * 2;
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1469: RuntimeWarning: divide by zero encountered in divide
  DyMadmin = np.ones(Dy.shape) / OverSampleRatio / SearchLimitY * 2;
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1413: RuntimeWarning: All-NaN slice encountered
  D = np.nanmedian(B,axis=0)
/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/isce/components/contrib/geo_autoRIFT/autoRIFT/autoRIFT.py:1416: RuntimeWarning: All-NaN slice encountered
  C = np.nanmedian(D,axis=0, overwrite_input=True)
/mnt/Backups/gbrench/sw/miniconda3/e

autoRIFT complete


/mnt/Backups/gbrench/sw/miniconda3/envs/insar/lib/python3.11/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


--------------------------------


In [11]:
# dem_fn = f'{proc_path}/work/merged/geom_reference/hgt.rdr.full'
# dem_ds = gdal.Open(dem_fn, gdal.GA_ReadOnly)
# dem = dem_ds.GetRasterBand(1).ReadAsArray()

In [12]:
# f, ax = plt.subplots(2, 1, figsize=(10, 10))
# ax[0].imshow(np.flip(obj.Dx_m, (0))[0:400, 400:1000], cmap='RdBu_r', vmin=-10, vmax=10, interpolation='none')
# ax[0].set_title(f'slant range offsets, {pairs[14][0]}-{pairs[14][1]}')
# ax[0].set_aspect(1)

# ax[1].imshow(np.flip(dem, (0))[0:1500, 10000:25000], vmin=3000, vmax=8000)
# ax[1].set_title(f'COP30 DEM')
# ax[1].set_aspect(6.13)

# f.tight_layout()